### For loops to pull L2A, L2B, L4A as needed

In [1]:
!pip install geopandas
!pip install contextily

  Using cached contextily-1.6.2-py3-none-any.whl.metadata (2.9 kB)
Using cached contextily-1.6.2-py3-none-any.whl (17 kB)


In [2]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.maap-project.org')

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
   
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

def wait_for_job(job_id: str) -> str:
    return job_status_for(job_id)

In [3]:
#pip install geopandas
#pip install shapely
#Import numpy for making column inputs easier
import numpy as np
import geopandas as gpd
import backoff
import shapely
import glob
import os

In [4]:
import xml.etree.ElementTree as ET
from urllib.parse import urlparse

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
    
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

In [5]:
# # Define the folder path containing the tiles
# #folder_path = "/projects/my-public-bucket/AOIs/GHANA/"
# folder_path = "/projects/my-public-bucket/Data/NASA_CMS_2023/WA_15_countries/"

# # Create a list to store the GeoPackage file paths
# json_files = []

# # Use glob to find all .gpkg files in the folder
# #pattern = os.path.join(folder_path, '*.geojson')
# pattern = os.path.join(folder_path, 'Mali_South.gpkg')
# json_files = glob.glob(pattern)

# # Print the list of GeoPackage file paths
# #for file_path in gpkg_files:
#  #   print(file_path)

# #json_files = [x.replace('/projects/my-public-bucket/AOIs/GHANA/',
# #                        'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/GHANA/') for x in json_files]
# json_files = [x.replace('/projects/my-public-bucket/Data/NASA_CMS_2023/WA_15_countries/',
#                         'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/WA_15_countries/') for x in json_files]

# print(len(json_files))
# print(json_files)
# AOIs = json_files
# AOIs[0]

In [6]:
# # OPTIONAL # visually verify the AOI
# aoi = AOIs[0]

# try:
#     import geopandas as gpd
#     import contextily as ctx
# except:
#     print(
#         "If you wish to visually verify your AOI, "
#         "you must install the `geopandas` and `contextily` packages."
#     )
# else:
#     aoi_gdf = gpd.read_file(aoi)
#     aoi_epsg4326 = aoi_gdf.to_crs(epsg=4326)
#     ax = aoi_epsg4326.plot(figsize=(10, 5), alpha=0.3, edgecolor="red")
#     ctx.add_basemap(ax, crs=4326)

In [7]:
# aoi_name = os.path.basename(AOIs[0]).split('.')[0]
# aoi_name

In [8]:
# folder_path = "/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/"
# json_files = []
# pattern = os.path.join(folder_path, '*.gpkg')
# json_files = glob.glob(pattern)
# json_files = [ x for x in json_files if "L2A" not in x ]
# json_files = [x.replace('/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/',
#                         'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MONGOLIA/GRID/') for x in json_files]
# print(len(json_files))
# print(json_files)
# AOIs = json_files
# AOIs[0]

folder_path = "/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/"
json_files = []
pattern = os.path.join(folder_path, '*.gpkg')
json_files = glob.glob(pattern)
json_files = [ x for x in json_files if "L2A" not in x ]
json_files = [x.replace('/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/',
                        'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/SUDAN/') for x in json_files]
print(len(json_files))
print(json_files)
AOIs = json_files
AOIs[0]

55
['https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/SUDAN/fid_1.gpkg', 'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/SUDAN/fid_10.gpkg', 'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/SUDAN/fid_12.gpkg', 'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/SUDAN/fid_14.gpkg', 'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/SUDAN/fid_15.gpkg', 'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/SUDAN/fid_17.gpkg', 'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/SUDAN/fid_19.gpkg', 'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/SUDAN/fid_2.gpkg', 'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/SUDAN/fid_20.gpkg', 'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/SUDAN/fid_21.gpkg'

'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/SUDAN/fid_1.gpkg'

In [9]:
#Set up run to pull products for all GEDI products simultaneously #To run for only 1 or 2 products, change "PRODUCTS"

PRODUCTS = ["L2A","L4A"] # ,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"

AOIs = json_files
#AOIs = [AOIs[2]]

for each_aoi in AOIs: 
    if "L4A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L4A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L4A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lat_lowestmode,lon_lowestmode,elev_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,sensitivity,geolocation/sensitivity_a2",
           query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95", #  and geolocation/sensitivity_a2 > 0.95",
           limit = 100000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_SUD_L4A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.8.0",
            queue="maap-dps-worker-32vcpu-64gb",
            username="nehajo88",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

    if "L2B" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2B subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2B",
           lat="geolocation/lat_lowestmode",
           lon="geolocation/lon_lowestmode",
           beams="all",
           columns="shot_number,geolocation/lon_lowestmode,geolocation/lat_lowestmode,rh100,l2b_quality_flag,sensitivity,cover,land_cover_data/landsat_treecover, pai,fhd_normal,"+",".join(variables),
           query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
           limit = 100000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_SUD_L2B.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.8.0",
            queue="maap-dps-worker-32vcpu-64gb",
            username="nehajo88",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result
        
        
    if "L2A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           #columns="shot_number,lon_lowestmode,lat_lowestmode,rh25,rh50,rh75,rh90,rh98,geolocation/sensitivity_a2",
           columns="shot_number,lon_lowestmode,lat_lowestmode,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80,rh90,rh98,quality_flag,rx_processing_a2/rx_algrunflag,rx_processing_a2/zcross,rx_processing_a2/toploc,geolocation/sensitivity_a2,degrade_flag,land_cover_data/pft_class,land_cover_data/region_class,surface_flag,rx_assess/rx_maxamp,rx_assess/sd_corrected,geolocation/stale_return_flag,land_cover_data/leaf_off_flag,land_cover_data/landsat_water_persistence,land_cover_data/urban_proportion",
           #columns="all",
           query="quality_flag == 1",
           limit = 100000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_SUD_L2A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.8.0",
            queue="maap-dps-worker-32gb",
            username="nehajo88",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result
        

Thanks, I'll run the GEDI L4A subsetter for fid_1!!
Thanks, I'll run the GEDI L2A subsetter for fid_1!!
Thanks, I'll run the GEDI L4A subsetter for fid_10!!
Thanks, I'll run the GEDI L2A subsetter for fid_10!!
Thanks, I'll run the GEDI L4A subsetter for fid_12!!
Thanks, I'll run the GEDI L2A subsetter for fid_12!!
Thanks, I'll run the GEDI L4A subsetter for fid_14!!
Thanks, I'll run the GEDI L2A subsetter for fid_14!!
Thanks, I'll run the GEDI L4A subsetter for fid_15!!
Thanks, I'll run the GEDI L2A subsetter for fid_15!!
Thanks, I'll run the GEDI L4A subsetter for fid_17!!
Thanks, I'll run the GEDI L2A subsetter for fid_17!!
Thanks, I'll run the GEDI L4A subsetter for fid_19!!
Thanks, I'll run the GEDI L2A subsetter for fid_19!!
Thanks, I'll run the GEDI L4A subsetter for fid_2!!
Thanks, I'll run the GEDI L2A subsetter for fid_2!!
Thanks, I'll run the GEDI L4A subsetter for fid_20!!
Thanks, I'll run the GEDI L2A subsetter for fid_20!!
Thanks, I'll run the GEDI L4A subsetter for fid_21

In [17]:
import subprocess

root_dir = "/projects/my-private-bucket/dps_output/gedi-subset/0.8.0/gedi-subset/2024/09/"  # Can now set name
out_dir = "/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/"

for dirName, subdirList, fileList in os.walk(root_dir):
#    print("Found directory: %s" % dirName)
    for fname in fileList:
        if fname.endswith("_SUD_L2A.gpkg") or fname.endswith("_SUD_L4A.gpkg"):
#        if fname.endswith("NPAs_L4A.gpkg"):
            source_path = os.path.join(dirName, fname)
            print(source_path)
            target_path = os.path.join(out_dir, fname)
            print(target_path)
            if not os.path.exists(target_path):  # Check if file doesn’t exist in target directory
                print(f"Moving {fname}...")
                subprocess.call(['mv', source_path, target_path])
            else:
                print(f"{fname} already exists in the target directory. Skipping...")

/projects/my-private-bucket/dps_output/gedi-subset/0.8.0/gedi-subset/2024/09/29/18/33/22/158972/fid_17_MON_L4A.gpkg
/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/fid_17_MON_L4A.gpkg
Moving fid_17_MON_L4A.gpkg...
/projects/my-private-bucket/dps_output/gedi-subset/0.8.0/gedi-subset/2024/09/29/18/33/25/673147/fid_18_MON_L4A.gpkg
/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/fid_18_MON_L4A.gpkg
Moving fid_18_MON_L4A.gpkg...
/projects/my-private-bucket/dps_output/gedi-subset/0.8.0/gedi-subset/2024/09/29/18/33/40/371740/fid_12_MON_L4A.gpkg
/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/fid_12_MON_L4A.gpkg
Moving fid_12_MON_L4A.gpkg...
/projects/my-private-bucket/dps_output/gedi-subset/0.8.0/gedi-subset/2024/09/29/18/33/43/055484/fid_10_MON_L4A.gpkg
/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/fid_10_MON_L4A.gpkg
Moving fid_10_MON_L4A.gpkg...
/projects/my-private-bucket/dps_output/gedi-subset/0.8.0/gedi-subset/2024/09/29/18/33/58/303158/

In [18]:
import subprocess

root_dir = "/projects/my-private-bucket/dps_output/gedi-subset/0.8.0/gedi-subset/2024/09/"  # Can now set name
out_dir = "/projects/my-public-bucket/Data/NASA_CMS_2023/WA_15_countries/GRID_Mali/"

for dirName, subdirList, fileList in os.walk(root_dir):
#    print("Found directory: %s" % dirName)
    for fname in fileList:
        if fname.endswith("_L2A.gpkg") or fname.endswith("_L4A.gpkg"):
#        if fname.endswith("NPAs_L4A.gpkg"):
            source_path = os.path.join(dirName, fname)
            print(source_path)
            target_path = os.path.join(out_dir, fname)
            print(target_path)
            if not os.path.exists(target_path):  # Check if file doesn’t exist in target directory
                print(f"Moving {fname}...")
                subprocess.call(['mv', source_path, target_path])
            else:
                print(f"{fname} already exists in the target directory. Skipping...")

/projects/my-private-bucket/dps_output/gedi-subset/0.8.0/gedi-subset/2024/09/29/18/46/52/556424/fid_11_L4A.gpkg
/projects/my-public-bucket/Data/NASA_CMS_2023/WA_15_countries/GRID_Mali/fid_11_L4A.gpkg
Moving fid_11_L4A.gpkg...
/projects/my-private-bucket/dps_output/gedi-subset/0.8.0/gedi-subset/2024/09/29/18/46/52/790671/fid_16_L4A.gpkg
/projects/my-public-bucket/Data/NASA_CMS_2023/WA_15_countries/GRID_Mali/fid_16_L4A.gpkg
Moving fid_16_L4A.gpkg...
/projects/my-private-bucket/dps_output/gedi-subset/0.8.0/gedi-subset/2024/09/29/18/46/56/003904/fid_14_L4A.gpkg
/projects/my-public-bucket/Data/NASA_CMS_2023/WA_15_countries/GRID_Mali/fid_14_L4A.gpkg
Moving fid_14_L4A.gpkg...
/projects/my-private-bucket/dps_output/gedi-subset/0.8.0/gedi-subset/2024/09/29/18/46/56/272845/fid_13_L4A.gpkg
/projects/my-public-bucket/Data/NASA_CMS_2023/WA_15_countries/GRID_Mali/fid_13_L4A.gpkg
Moving fid_13_L4A.gpkg...
/projects/my-private-bucket/dps_output/gedi-subset/0.8.0/gedi-subset/2024/09/29/18/46/57/118196/